## <center>Курсовой проект</center>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_curve
import dill

Загрузим датасет, сделаем разбиение на тренировочную и тестовую выборки 

In [2]:
df = pd.read_csv("/home/maxim/Документы/GeekBrains/4 четверть. Машинное обучение в бизнесе/Tasks/CR/churn_data.csv")
df.drop(columns='CustomerId', inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=42)

In [3]:
y_test.to_csv('y_test.csv', index=False, sep=';')
X_test.to_csv('X_test.csv', index=False, sep=';')

In [4]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

Определим колонки, которые отнесем к категориальным и к численным.
Для категориальных применим OHE, а численные стандартизируем.

In [5]:
features = [
    'Geography', 
    'Gender', 
    'Tenure', 
    'HasCrCard', 
    'IsActiveMember', 
    'CreditScore', 
    'Age', 
    'Balance', 
    'NumOfProducts', 
    'EstimatedSalary'
]

target = 'Exited'

In [6]:
geography = Pipeline([
    ('selector', FeatureSelector(column='Geography')),
    ('ohe', OHEEncoder(key='Geography')),
])
gender = Pipeline([
    ('selector', FeatureSelector(column='Gender')),
    ('ohe', OHEEncoder(key='Gender')),
])
tenure = Pipeline([
    ('selector', FeatureSelector(column='Tenure')),
    ('ohe', OHEEncoder(key='Tenure')),
])
hascrcard = Pipeline([
    ('selector', FeatureSelector(column='HasCrCard')),
    ('ohe', OHEEncoder(key='HasCrCard')),
])
isactivemember = Pipeline([
    ('selector', FeatureSelector(column='IsActiveMember')),
    ('ohe', OHEEncoder(key='IsActiveMember')),
])

creditscore = Pipeline([
                ('selector', NumberSelector(key='CreditScore')),
                ('stand', StandardScaler()),
            ])
age = Pipeline([
                ('selector', NumberSelector(key='Age')),
                ('stand', StandardScaler()),
            ])
balance = Pipeline([
                ('selector', NumberSelector(key='Balance')),
                ('stand', StandardScaler()),
            ])
numofproducts = Pipeline([
                ('selector', NumberSelector(key='NumOfProducts')),
                ('stand', StandardScaler()),
            ])
estimatedsalary = Pipeline([
                ('selector', NumberSelector(key='EstimatedSalary')),
                ('stand', StandardScaler()),
            ])

Объединим всё это в единый пайплайн

In [7]:
feats = FeatureUnion([
    ('Geography', geography),
    ('Gender', gender),
    ('Tenure', tenure),
    ('HasCrCard', hascrcard),
    ('IsActiveMember', isactivemember),
    ('CreditScore', creditscore),
    ('Age', age),
    ('Balance', balance),
    ('NumOfProducts', numofproducts),
    ('EstimatedSalary', estimatedsalary),
])

# feature_processing = Pipeline([('feats', feats)])

Обучим модель с градиентным бустингом

In [8]:
model = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42)),
])

In [9]:
model.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [10]:
preds_gb = model.predict_proba(X_test)[:, 1]

In [11]:
precision_gb, recall_gb, thresholds_gb = precision_recall_curve(y_test, preds_gb)

fscore_gb = (2 * precision_gb * recall_gb) / (precision_gb + recall_gb)

ix_gb = np.argmax(fscore_gb)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_gb[ix_gb], 
                                                                        fscore_gb[ix_gb],
                                                                        precision_gb[ix_gb],
                                                                        recall_gb[ix_gb]))

Best Threshold=0.271906, F-Score=0.642, Precision=0.602, Recall=0.686


In [12]:
result_table = pd.DataFrame({
    'Gradient Boosting Model': pd.Series({
        'Thresholds': thresholds_gb[ix_gb], 
        'FScore': fscore_gb[ix_gb], 
        'Precision': precision_gb[ix_gb], 
        'Recall': recall_gb[ix_gb]
    }),
})

result_table

,Gradient Boosting Model
Thresholds,0.271906
FScore,0.641580
Precision,0.602473
Recall,0.686117


In [13]:
with open("app/gradboost_pipeline.dill", "wb") as f:
    dill.dump(model, f)